# Toy problem for debugging Dual Decomposition

$$
\begin{align*}
\max_{x_k,y} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y \leq d_k, \\
& y\in \{0,1\}.
\end{align*}
$$

$$
\begin{align*}
\max_{x_k,y_k} \quad & \sum_k f_k'y + \sum_k c_k'x_k \\
\mathrm{s.t.} \quad & A_k x_k + B_k y_k \leq d_k, \\
& \sum_k H_k y_k = 0, \\
& y_k\in \{0,1\}.
\end{align*}
$$

In [1]:
RELAX_LP = True

In [2]:
from __future__ import division
from cobra import Model
from dynamicme.optimize import Variable, Constraint
from six import iteritems
import numpy as np

### Example from
http://www.optirisk-systems.com/manuals/FortspManual.pdf

$$
\begin{align*}
\max_{x,y} \quad & \sum_k P_k \left( \sum_c p^s_c x^s_{c,k} - p^b_c x^b_{c,k}  - \sum_c c_c a_c \right)\\
\mathrm{s.t.} \quad & \sum_c a_c \leq A \\
& Y_{c,k}  a_c - x^s_{c,k} + x^b_{c,k} \geq R_c \\
& x^s_{\mathrm{beet},k} \leq Q(\mathrm{beet}) \\
& x^s_{\mathrm{beet},c} + s^x_k \leq Y_{\mathrm{beet},k} a_\mathrm{beet}
\end{align*}
$$

In [3]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

In [4]:
#------------------------------------------------------------
# Data
crops = ['wheat','corn','beet']
total_area = 500.  # Total acres
prob_dict = {'below':1./3, 'average':1./3, 'above':1./3}
# Yields (tons/acre) over scenarios
yield_dict = {}
yield_dict['below'] = {'wheat':2., 'corn':2.4, 'beet':16.}
yield_dict['average'] = {'wheat':2.5, 'corn':3., 'beet':20.}
yield_dict['above'] = {'wheat':3., 'corn':3.6, 'beet':24.}
# Planting costs
plantcost_dict={'wheat':150, 'corn':230, 'beet':260}   # $/ton
sellprice_dict = {'wheat':170, 'corn':150, 'beet':36}  # $/ton
excess_sell_price = 10    # beet sell price ($/ton) when quota exceeded
beet_quota = 6000   # tons
buyprice_dict = {'wheat':238, 'corn':210, 'beet':100}  # $/ton
req_dict = {'wheat':200., 'corn':240., 'beet':0.} # tons required to feed cattle
#------------------------------------------------------------
# Area is complicating (first-stage) variable

In [5]:
# Create subproblems
UB = 1e4
sub_dict = {}
for scen,yields in iteritems(yield_dict):
    # Each subproblem gets its own copy of first-stage variables
    area_dict = {c:Variable('area_%s'%c, lower_bound=0., upper_bound=1e6,
        objective_coefficient=prob_dict[scen]*plantcost_dict[c]) for c in crops}
    for y in area_dict.values():
        y.variable_kind = 'integer'
    mdl = Model('sub')
    mdl.add_reactions(area_dict.values())
    # Global constraint: sum_a <= Area
    cons_area = Constraint('total_area')
    cons_area._bound = total_area
    cons_area._constraint_sense = 'L'
    mdl.add_metabolites(cons_area)
    for y in area_dict.values():
        y.add_metabolites({cons_area:1.}, combine=False)

    # MAX sell price
    x_excess = Variable('sell_excess_beet', lower_bound=0., upper_bound=UB)
    x_excess.objective_coefficient = -prob_dict[scen]*excess_sell_price    
    mdl.add_reaction(x_excess)
    
    for c in crops:
        ### Scenario variables x: tons sold, bought, excess sold per scenario
        x_sell = Variable('sell_%s'%(c), lower_bound=0., upper_bound=UB)
        x_buy  =  Variable('buy_%s'%(c), lower_bound=0., upper_bound=UB)
        mdl.add_reactions([x_sell,x_buy])        
        x_sell.objective_coefficient = -prob_dict[scen]*sellprice_dict[c]
        x_buy.objective_coefficient = prob_dict[scen]*buyprice_dict[c]
        
        ### Scenario constraint: Y_ck*ac - xs_ck + xb_ck >= Req_c        
        cons = Constraint('required_%s'%c)
        cons._bound = req_dict[c]
        cons._constraint_sense = 'G'        
        mdl.add_metabolites(cons)
        area = area_dict[c]
        area.add_metabolites({cons:yield_dict[scen][c]})
        x_sell.add_metabolites({cons:-1.})
        x_buy.add_metabolites({cons:1.})
        
        ### Beet quota
        if c=='beet':
            x_sell.upper_bound = beet_quota
            # xs_beet,k + xexcess_k <= Yield_beet,k * area_beet
            cons_beet = Constraint('beet_balance')
            cons_beet._bound = 0.
            cons_beet._constraint_sense = 'L'
            x_sell.add_metabolites({cons_beet:1.})
            x_excess.add_metabolites({cons_beet:1.})
            area_dict[c].add_metabolites({cons_beet:-yield_dict[scen][c]})
    
    sub = LagrangeSubmodel(mdl, scen)
    sub_dict[scen] = sub

In [6]:
master = LagrangeMaster(mdl)
master._INF = 1e7
master.add_submodels(sub_dict)
master._z.LB = -master._INF
master._z.UB = master._INF
master.model.update()

In [7]:
obj = master.model.getObjective()
obj

<gurobi.QuadExpr: z + [ -0.5 u_00 ^ 2 + -0.5 u_01 ^ 2 + -0.5 u_02 ^ 2 + -0.5 u_10 ^ 2 + -0.5 u_11 ^ 2 + -0.5 u_12 ^ 2 ]>

In [8]:
for v in master.model.getVars():
    print(v.VarName, v.LB, v.UB)

('z', -10000000.0, 10000000.0)
('tk_below', -10000000.0, 10000000.0)
('tk_average', -10000000.0, 10000000.0)
('tk_above', -10000000.0, 10000000.0)
('u_00', -10000000.0, 10000000.0)
('u_01', -10000000.0, 10000000.0)
('u_02', -10000000.0, 10000000.0)
('u_10', -10000000.0, 10000000.0)
('u_11', -10000000.0, 10000000.0)
('u_12', -10000000.0, 10000000.0)


In [9]:
for sub_ind,sub in iteritems(sub_dict):
    print('H_%s'%sub_ind)
    print(sub._H.todense())

H_below
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
H_average
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
H_above
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [10]:
cc = master.model.getConstrs()[0]
print(master.model.getRow(cc), cc.Sense, cc.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [11]:
master.model.getObjective()

<gurobi.QuadExpr: z + [ -0.5 u_00 ^ 2 + -0.5 u_01 ^ 2 + -0.5 u_02 ^ 2 + -0.5 u_10 ^ 2 + -0.5 u_11 ^ 2 + -0.5 u_12 ^ 2 ]>

In [12]:
cz = master.model.getConstrByName('z_cut')
print(master.model.getRow(cz), cz.Sense, cz.RHS)

(<gurobi.LinExpr: z + -1.0 tk_below + -1.0 tk_average + -1.0 tk_above>, '<', 0.0)


In [13]:
from dynamicme.callback_gurobi import cb_benders_multi

In [14]:
# Need to scale variables? Since values are huge?

In [15]:
master.model.Params.Presolve = 0
# master.model.Params.ScaleFlag = 2
# master.model.Params.NumericFocus = 3
master.model.Params.BarHomogeneous = 1

# master.model.Params.Method = 0  # primal simplex
# master.model.Params.Method = 1  # dual simplex
master.model.Params.Method = 2  # barrier

master.model.Params.OutputFlag=0
master.max_iter = 100
master.print_iter=1
master.verbosity = 0

master.delta_mult = 0.5   #0.1
master.delta_min  = 1e-20 #0

if RELAX_LP:
    uopt = master.solve_relaxed()
else:    
    uopt = master.optimize() 

        Iter          UB          LB    Best Bnd         gap   relgap(%)       delta     time(s)
           0       1e+07  -1.154e+05  -1.154e+05   1.012e+07       101.2           1    0.006040
           1  -8.393e+04  -1.973e+05  -1.154e+05   3.148e+04       37.51         0.5    0.010682
           2  -8.801e+04  -2.377e+05  -1.154e+05    2.74e+04       31.13        0.25    0.015229
           3  -1.024e+05  -1.292e+05  -1.154e+05   1.303e+04       12.73       0.125    0.019704
           4  -1.067e+05  -1.103e+05  -1.103e+05        3560       3.337      0.0625    0.024486
           5  -1.077e+05  -1.128e+05  -1.103e+05        2571       2.388     0.03125    0.028990
           6  -1.084e+05  -1.084e+05  -1.084e+05   2.419e-07   2.232e-10     0.01562    0.033591
relgap (2.23213e-12) <= gaptol (0.0001). Finished.
sum_k Hk*yk: [ 300.           20.         -320.         -250.         -103.33333333
  353.33333333]


In [16]:
print(master._z.X)
master.model.getObjective()

-108390.000001


<gurobi.QuadExpr: z + [ -0.0078125 u_00 ^ 2 + -0.0078125 u_01 ^ 2 + -0.0078125 u_02 ^ 2 + -0.0078125 u_10 ^ 2 + -0.0078125 u_11 ^ 2 + -0.0078125 u_12 ^ 2 ]>

In [17]:
master.model

<gurobi.Model Continuous instance master: 19 constrs, 10 vars, Parameter changes: Method=2, BarHomogeneous=1, LogFile=gurobi.log, Presolve=0, OutputFlag=0>

In [18]:
cons = master.model.getConstrs()[1]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[2]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[3]
print(master.model.getRow(cons), cons.Sense, cons.RHS)

(<gurobi.LinExpr: tk_below + -375.0 u_00 + -25.0 u_01 + -100.0 u_02>, '<', -19983.333333333336)
(<gurobi.LinExpr: tk_average + 300.0 u_00 + 80.0 u_01 + 120.0 u_02 + -300.0 u_10 + -80.0 u_11 + -120.0 u_12>, '<', -39533.33333333334)
(<gurobi.LinExpr: tk_above + 250.0 u_10 + 66.6666666667 u_11 + 183.333333333 u_12>, '<', -55888.88888888889)


In [19]:
cons = master.model.getConstrs()[4]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[5]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[6]
print(master.model.getRow(cons), cons.Sense, cons.RHS)

(<gurobi.LinExpr: tk_below + -500.0 u_01>, '<', 6199.999999999993)
(<gurobi.LinExpr: tk_average + 300.0 u_00 + 200.0 u_02 + -300.0 u_10 + -200.0 u_12>, '<', -36200.0)
(<gurobi.LinExpr: tk_above + 250.0 u_10 + 250.0 u_11>, '<', -48300.000000000015)


In [20]:
ncons = len(master.model.getConstrs())
cons = master.model.getConstrs()[ncons-3]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[ncons-2]
print(master.model.getRow(cons), cons.Sense, cons.RHS)
cons = master.model.getConstrs()[ncons-1]
print(master.model.getRow(cons), cons.Sense, cons.RHS)

(<gurobi.LinExpr: tk_below + -300.0 u_00 + -100.0 u_01 + -100.0 u_02>, '<', -18933.333333333336)
(<gurobi.LinExpr: tk_average + 80.0 u_01 + 420.0 u_02 + -80.0 u_11 + -420.0 u_12>, '<', -21033.333333333332)
(<gurobi.LinExpr: tk_above + 250.0 u_10 + 183.333333333 u_11 + 66.6666666667 u_12>, '<', -53944.444444444445)


## Check answer: should be 
- {'wheat':170, 'corn':80, 'beet':250}
- objval = (-)108390

In [21]:
master._z

<gurobi.Var z (value -108390.000001)>

In [22]:
x_dict = {}
for scen,sub in iteritems(master.sub_dict):
    #sub.update_obj(yopt)
    #sub.model.optimize()
    #x_dict[scen] = {v.VarName:v.X for v in sub.model.getVars()}
    x_dict[scen] = sub.x_dict

In [23]:
x_dict

{'above': {'area_beet': 250.0,
  'area_corn': 183.33333333333334,
  'area_wheat': 66.66666666666667,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 6000.0,
  'sell_corn': 420.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 0.0},
 'average': {'area_beet': 0.0,
  'area_corn': 80.0,
  'area_wheat': 420.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 0.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 850.0},
 'below': {'area_beet': 300.0,
  'area_corn': 100.0,
  'area_wheat': 100.0,
  'buy_beet': 0.0,
  'buy_corn': 0.0,
  'buy_wheat': 0.0,
  'sell_beet': 4800.0,
  'sell_corn': 0.0,
  'sell_excess_beet': 0.0,
  'sell_wheat': 0.0}}

In [24]:
Hys = []
sub_objs = []
for sub_ind,sub in iteritems(sub_dict):
    yk = np.array([sub.x_dict[x.VarName] for x in sub._ys])
    Hyk = sub._H*yk
    Hys.append(Hyk)    
    sub_objs.append(sub.model.ObjVal)
    print('Scenario: %s'%sub_ind)
    print('yk = %s'%[(k,v) for k,v in iteritems(sub.x_dict)])
    print('Hk*yk =%s' % Hyk)
    
print("\nsum_k Hk*yk=")
print(sum(Hys))
print("sum sum Hk*yk=%s"%sum(sum(Hys)))
print("\n u = %s" % master.uopt)

Scenario: below
yk = [('buy_wheat', 0.0), ('area_beet', 300.0), ('area_corn', 100.0), ('sell_wheat', 0.0), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 0.0), ('sell_beet', 4800.0), ('buy_beet', 0.0), ('area_wheat', 100.0)]
Hk*yk =[300. 100. 100.   0.   0.   0.]
Scenario: average
yk = [('buy_wheat', 0.0), ('area_beet', 0.0), ('area_corn', 80.0), ('sell_wheat', 850.0), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 0.0), ('sell_beet', 0.0), ('buy_beet', 0.0), ('area_wheat', 420.0)]
Hk*yk =[   0.  -80. -420.    0.   80.  420.]
Scenario: above
yk = [('buy_wheat', 0.0), ('area_beet', 250.0), ('area_corn', 183.33333333333334), ('sell_wheat', 0.0), ('buy_corn', 0.0), ('sell_excess_beet', 0.0), ('sell_corn', 420.0), ('sell_beet', 6000.0), ('buy_beet', 0.0), ('area_wheat', 66.66666666666667)]
Hk*yk =[   0.            0.            0.         -250.         -183.33333333
  -66.66666667]

sum_k Hk*yk=
[ 300.           20.         -320.         -250.         -103.3333333

In [25]:
Hys = []
sub_objs = []
cxs = []
fys = []
for sub_ind,sub in iteritems(master.sub_dict):
    xk = np.array([sub.x_dict[x.VarName] for x in sub._xs])
    yk = np.array([sub.x_dict[x.VarName] for x in sub._ys])
    Hyk = sub._H*yk
    Hys.append(Hyk)    
    sub_objs.append(sub.model.ObjVal)
    cx = np.dot(sub._cx, xk)
    fy = np.dot(sub._fy, yk)
    cxs.append(cx)
    fys.append(fy)
    print('Scenario: %s'%sub_ind)
    print('c*x = %s' % cx)
    print('f*y = %s' % fy)
    print('cx + fy = %s'%(cx+fy))    
    uHy = np.dot(master.uopt, sub._H*yk)
    print("u*Hk*yk = %s" % (uHy))
    print('Sub obj val: %s' % sub.model.ObjVal)    
    print('-'*40)

# print("\nsum_k Hk*yk=")
# print(sum(Hys))
# print("sum sum Hk*yk=%s"%sum(sum(Hys)))
# print("\n u = %s" % master.uopt)
print("sum_k sub obj val: %s" % sum(sub_objs))
print("sum_k cx + fy = %s" % (sum(cxs)+sum(fys)))

Scenario: below
c*x = -57600.0
f*y = 38666.666666666664
cx + fy = -18933.333333333336
u*Hk*yk = 3733.333334326744
Sub obj val: -15199.999999
----------------------------------------
Scenario: average
c*x = -48166.666666666664
f*y = 27133.333333333332
cx + fy = -21033.333333333332
u*Hk*yk = -9295.555557645857
Sub obj val: -30328.888891
----------------------------------------
Scenario: above
c*x = -93000.0
f*y = 39055.555555555555
cx + fy = -53944.444444444445
u*Hk*yk = -8916.666666759796
Sub obj val: -62861.1111112
----------------------------------------
sum_k sub obj val: -108390.000001
sum_k cx + fy = -93911.1111111111


In [26]:
for sub_ind,sub in iteritems(master.sub_dict):
    sub.update_obj(master.uopt)
    sub.optimize()
    print('-'*40)
    print('Scenario: %s'%sub_ind)
#     print('f: %s' % sub._fy)
    print('yk = %s' % {v.VarName:v.X for v in sub._ys})
#     print('xk = %s' % {v.VarName:v.X for v in sub._xs})

----------------------------------------
Scenario: below
yk = {'area_beet': 0.0, 'area_corn': -0.0, 'area_wheat': 500.0}
----------------------------------------
Scenario: average
yk = {'area_beet': -0.0, 'area_corn': 80.0, 'area_wheat': 420.0}
----------------------------------------
Scenario: above
yk = {'area_beet': 250.0, 'area_corn': 183.0, 'area_wheat': 67.0}


In [27]:
uk = master.uopt 
uk

array([ 18.66666667,   4.66666665, -23.33333333,  63.55555556,
       -23.44444444, -40.11111112])

In [28]:
sub = sub_dict['average']
sub._H.todense()

matrix([[-1.,  0.,  0.],
        [ 0., -1.,  0.],
        [ 0.,  0., -1.],
        [ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]])

In [29]:
master._us

[<gurobi.Var u_00 (value 18.6666666735)>,
 <gurobi.Var u_01 (value 4.66666665487)>,
 <gurobi.Var u_02 (value -23.3333333321)>,
 <gurobi.Var u_10 (value 63.555555556)>,
 <gurobi.Var u_11 (value -23.4444444422)>,
 <gurobi.Var u_12 (value -40.1111111175)>]

In [30]:
uk*sub._H

array([ 44.88888888, -28.1111111 , -16.77777779])

In [31]:
from gurobipy import LinExpr
LinExpr(uk*sub._H, sub._ys)

<gurobi.LinExpr: 44.8888888825 area_beet + -28.111111097 area_corn + -16.7777777854 area_wheat>

## Check that all constraints satisfied

In [32]:
# Total area
for sub_ind,sub in iteritems(sub_dict):
    yopt = [v.X for v in sub._ys]
    print('Scenario=%s. Total area constraint:'%sub_ind, sum(yopt) <= total_area)

('Scenario=below. Total area constraint:', True)
('Scenario=average. Total area constraint:', True)
('Scenario=above. Total area constraint:', True)


In [33]:
# Cattle feed crop requirement
print("Cattle feed crop requirement met?\n%s"%('-'*40))
for scen,sub in iteritems(sub_dict):
    y_dict = {v.VarName:v.X for v in sub._ys}
    for c,req in iteritems(req_dict):
        area = y_dict['area_%s'%c]
        xsell = x_dict[scen]['sell_%s'%(c)]
        xbuy = x_dict[scen]['buy_%s'%(c)]
        lhs = yield_dict[scen][c]*area - xsell + xbuy
        rhs = req
        print('%s, %s: %g >= %g, %s'%(scen,c,lhs,rhs,lhs>=rhs))

Cattle feed crop requirement met?
----------------------------------------
below, beet: -4800 >= 0, False
below, corn: 0 >= 240, False
below, wheat: 1000 >= 200, True
average, beet: 0 >= 0, True
average, corn: 240 >= 240, True
average, wheat: 200 >= 200, True
above, beet: 0 >= 0, True
above, corn: 238.8 >= 240, False
above, wheat: 201 >= 200, True
